In [5]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [6]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

# Loading Links & Creating DataFrame

In [7]:
input_filename = "links.txt"
df = text_file_to_dataframe(input_filename)

In [8]:
df.head(3)

,links
0,https://www.goodreads.com/book/show/13771831-b...
1,https://www.goodreads.com/book/show/28111953-j...
2,https://www.goodreads.com/book/show/24612758-m...


In [9]:
print(df['links'].iloc[2])

https://www.goodreads.com/book/show/24612758-millie-s-tropical-world?from_search=true&from_srp=true&qid=WtOqQXjVXW&rank=1


In [ ]:
#

In [22]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()


In [55]:
driver.get("https://www.goodreads.com/book/show/1618.The_Curious_Incident_of_the_Dog_in_the_Night_Time")

In [53]:
# find the element with the EditionDetails class
edition_details_element = driver.find_element_by_class_name('EditionDetails')
# find the nested element with the data-testid attribute
content_container = edition_details_element.find_elements_by_css_selector('div[data-testid="contentContainer"]')
print(content_container[1].text)
#publisher = content_container[1].text.split(" by ")[-1]
#print(publisher)

May 18, 2004 by Vintage


In [17]:
# Attempt to extract the publisher text
publisher_text = get_publisher_text()

# If the publisher text is not found, click the button and try again
if publisher_text is None:
    try:
        button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')
        button.click()
        
        # Wait for the publisher element to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails")))

        # Try to extract the publisher text again
        publisher_text = get_publisher_text()
    except NoSuchElementException:
        print("Button not found")

# Print the extracted publisher text
if publisher_text is not None:
    print(publisher_text)
else:
    print("Publisher text not found")

Ouput of publisher



# Scraper Code

In [54]:
# Define a function to extract the publisher element text
def get_publisher_text():
    try:
        # find the element with the EditionDetails class
        edition_details_element = driver.find_element_by_class_name('EditionDetails')
        
        # find the nested element with the data-testid attribute
        publisher_element = edition_details_element.find_elements_by_css_selector('div[data-testid="contentContainer"]')
        return publisher_element[1].text
    
    except NoSuchElementException:
        return None

In [58]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("reviews.csv")
print("Does Review CSV File Exists?")
print(reviews_file_exists)
print("\n")

index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print("Output of last_index")
print(last_index)

# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    print("Ouput of index")
    print(index)
    
    print("Ouput of row")
    print(row)

    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    try:
        print("Waiting for Title Element to Load....")
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'BookPageTitleSection__title')))
        print("Assigning Title")
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue
    
    print("Ouput of title")
    print(title)
    
    #============================================================================================================
    # P U B L I S H E R   D A T A
    #============================================================================================================
    # Attempt to extract the publisher text
    publisher_text = get_publisher_text()

    # If the publisher text is not found, click the button and try again
    if publisher_text is None:
        try:
            button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')
            try:
                button.click()

                # Wait for the publisher element to load
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails")))

                # Try to extract the publisher text again
                publisher_text = get_publisher_text()
            except Exception as e:
                print(f"Unable to click the button due to {e}")
        except NoSuchElementException:
            print("Button not found")

    # Print the extracted publisher text
    if publisher_text is not None:
        print(publisher_text)
    else:
        print("Publisher text not found")

    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        print("Waiting for Synopsis and Reviews to load....")
        time.sleep(20)
        synopsis_and_review_list = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Formatted')))
        print(f"The variable length of book {index+1} = ")
        print(len(synopsis_and_review_list))
        synopsis_length = len(synopsis_and_review_list)
        print("Output of synopsis_length")
        print(synopsis_length)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    
    try:
        page_count_element = driver.find_element_by_css_selector('p[data-testid="pagesFormat"]')
        page_count = page_count_element.text
        
        print("Ouput of page_count")
        print(page_count)
    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element_by_css_selector('p[data-testid="publicationInfo"]')
        year = year_element.text
        
        print("Ouput of year")
        print(year)
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element_by_css_selector('span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        print("Ouput of review_count")
        print(review_count)
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
            print("Ouput of RegEx: review_count")
            print(review_count)
        else:
            review_count = 0
            print("Ouput of No_RegEx: review_count")
            print(review_count)
            
    except NoSuchElementException:
        review_count = 0
        print("Ouput of No_Count: review_count")
        print(review_count)
    
    try:
        rating_count_element = driver.find_element_by_css_selector('span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text
        print("Ouput of rating_count")
        print(rating_count)
    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element_by_class_name('RatingStatistics__rating')    
        rating = rating_element.text
        print("Ouput of rating")
        print(rating)
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element_by_class_name('BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements_by_css_selector('.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        print("Ouput of genre")
        print(genre)
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element_by_class_name('ContributorLink__name')
        author = author_element.text
        print("Ouput of author")
        print(author)

    except NoSuchElementException:
        author = ""
        
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author,publisher]
    
    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'page_count'] = page_count
    df.at[index, 'publisher'] = publisher
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author

    
    # =====================
    # T E S T   P R I N T
    # =====================
    print(f"Book {index}:")
    print(title)
    print("==============")
    print(page_count)
    print("\n")
    print("==============")
    print(year)
    print("\n")
    print("==============")
    print("Total Reviews: ")
    print(review_count)
    print("\n")
    print("==============")
    print(rating)
    print("\n")
    print("==============")
    print(rating_count)
    print("\n")
    print("==============")
    print(genre)
    print("\n")
    print("==============")
    print(author)
    print("\n")
    print("==============")
    print(publisher)
    print("\n")
    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    review_index = 3
    for i in range(5):
        if review_index < synopsis_length:
            print(f"Review {i + 1}")
            print("==============")
            print(synopsis_and_review_list[review_index].text)

            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("reviews.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'publisher', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("reviews.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()

Does Review CSV File Exists?
True


Output of last_index
12
Ouput of index
12
Ouput of row
links           https://www.goodreads.com/book/show/24612758-m...
page_count                                                    NaN
publisher                                                     NaN
year                                                          NaN
review_count                                                  NaN
rating                                                        NaN
rating_count                                                  NaN
genre                                                         NaN
author                                                        NaN
Name: 12, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
Millie's Tropical World
September 8, 2015 by Lark Crafts
Waiting for Synopsis and Reviews to load....
The variable length of book 13 = 
5
Output of synopsis_length
5
Ouput of page_count
96 pages, Paperback
Ouput of year
Fi

KeyboardInterrupt: 